# Postgres Database Optimization Walk Through

## Database and Query Creation

Firstly, all necessary modules are imported:

In [2]:
import psycopg2
from postgres_data_insertion import insert
from postgres_optimization_measurements import measure_query_runtime, run_query

Prerequisities for running this file is to set up a Postgres database named `musicians` and create the tables running the file `postgres_create_tables.sql` in pgAdmin. After creating the database it is filled using the following command:

In [3]:
insert()

Inserted tuples into bands table.
Inserted tuples into albums table.
Inserted tuples into genres table.
Inserted tuples into musicians table.
Inserted tuples into has_genre table.
Inserted tuples into member_of table.
Inserted tuples into has_name table.
Filling of database with data done.


For data retrieval two complex queries were implemented in `postgres_queries.sql`. The tasks of the queries are listed respectively:

1. **Get the most successful band in the 2010s (01.01.2010 - 31.12.2019) in the most successful genre of the 1990s (01.01.1990 - 31.12.1999)**

2. **Add a new album to the most successful band of most successful genre in the 1990s, so that it is more successful than all of the albums of the most successful band in this genre in the 2010s**

For the next steps only the first query is considered as the computations in both queries are similar. The next cell runs the first query and outputs the result:

In [4]:
query_not_optimized='''SELECT get_most_succesful_band_in_timeframe_in_most_successful_genre_90s('31-12-2009'::date, '01-01-2020'::date);'''

with psycopg2.connect(dbname='musicians', user='postgres', password='root') as conn:
    cur = conn.cursor()
    cur.execute(query_not_optimized)
    res=cur.fetchone()[0]
res

'http://dbpedia.org/resource/Linkin_Park'

## Database Optimization

Implementation of the optimization steps of the Postgres database are in the file `postgres_optimization.sql` and the time measurements of each step in `postgres_optimization_measurements.py`. The stepwise evaluation process of the optimization is formally listed below:

Stages of optimization:
1. No optimization
2. Optimization by creating indexes on `albums.release_date` and `albums.band_urls`
3. Optimization by creating a materialized view on the common join operations
4. Optimization by indexing the materialized view of the most common joins
5. Optimization by precomputing the results of the 1. complex query and saving it as a materialized view

For each step of optimization the execution time of the query is computed by running a query `n_runs` times and calculating the mean for all runs:

In [5]:
query_drop_indexes='''SELECT drop_indexes();'''
query_create_indexes='''SELECT create_indexes();'''
query_create_index_view='''SELECT create_index_on_view();'''
query_not_optimized='''SELECT get_most_succesful_band_in_timeframe_in_most_successful_genre_90s('31-12-2009'::date, '01-01-2020'::date);'''
query_optimized_view='''SELECT get_most_succesful_band_in_timeframe_in_most_successful_genre_90s_optimzed_view_joins('31-12-2009'::date, '01-01-2020'::date);'''
query_fully_optimized='''SELECT get_most_succesful_band_in_timeframe_in_most_successful_genre_90s_fully_optimized()'''
    
n_runs=10
        
      
# 1. Runtime not optimized:
run_query(query_drop_indexes)
runtime_not_optimized=measure_query_runtime(query_not_optimized, n_runs)
print('Runtime not optimized: {}'.format(runtime_not_optimized))
    
    
# 2. Runtime with indexes:
run_query(query_create_indexes)
runtime_with_indexes=measure_query_runtime(query_not_optimized, n_runs)
print('Runtime with indexes: {}'.format(runtime_with_indexes))
    
# 3. Runtime with view on joins:
runtime_with_view=measure_query_runtime(query_optimized_view, n_runs)
print('Runtime with view on joins: {}'.format(runtime_with_view))
    
    
# 4. Runtime with index on view on joins:
run_query(query_create_index_view)
runtime_with_view_and_index=measure_query_runtime(query_optimized_view, n_runs)
print('Runtime with index on view on joins: {}'.format(runtime_with_view_and_index))
    
    
# 5. Runtime fully optimized query:
runtime_fully_optimized=measure_query_runtime(query_fully_optimized, n_runs)
print('Runtime fully optimized: {}'.format(runtime_fully_optimized))

Runtime not optimized: 0.0657968521118164
Runtime with indexes: 0.06960055828094483
Runtime with view on joins: 0.07008936405181884
Runtime with index on view on joins: 0.06545779705047608
Runtime fully optimized: 0.02109973430633545
